In [4]:
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [5]:
market = Market()
sec = SEC()

In [6]:
financial_db = ADatabase("strategy_financial")

In [92]:
market.connect()
sp5 = market.retrieve("sp500")
econ = market.retrieve("econ")
prices = market.retrieve("alpha_prices")
market.disconnect()
financial_db.connect()
sim = financial_db.retrieve("sim")
financial_db.disconnect()

In [93]:
prices = p.column_date_processing(prices)
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["year"] = [x.year for x in prices["date"]]
prices["close"] = [float(x) for x in prices["close"]]

In [121]:
ddm_sim = []
rr = 0.1
growth_rate = 0.05
sec.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        financials = filing.rename(columns={"filed":"date"})
        financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in financials["date"]]
        financials["year"] = [x.year for x in financials["date"]]
        financials["quarter"] = [x.quarter for x in financials["date"]]
        financials = p.column_date_processing(financials)
        labels = prices[prices["ticker"]==ticker][["year","quarter","ticker","close"]].groupby(["year","quarter","ticker"]).mean().reset_index()
        data = financials[["year","quarter","commonstockdividendspersharedeclared"]].merge(labels,on=["year","quarter"])
        ticker_sim = sim[sim["ticker"]==ticker].copy()
        ticker_sim["year"] = ticker_sim["year"] - 1
        ticker_sim["prediction"] = [sum([row[1][col] for col in ticker_sim.columns if "prediction" in col])/3 for row in ticker_sim.iterrows()]
        ddm_data = data.merge(ticker_sim,on=["year","quarter","ticker"]).groupby(["year","quarter"]).mean().reset_index()
        ddm_values = []
        for row in ddm_data.iterrows():
            dividend = row[1]["commonstockdividendspersharedeclared"]
            yearendprice = row[1]["prediction"]
            value = dividend * (1+growth_rate) /(1+rr) + yearendprice/(1+rr)
            ddm_values.append(value)
        ddm_data["ddm_value"] = ddm_values
        ddm_data = ddm_data.fillna(method="ffill")
        ddm_data["ticker"] = ticker
        ddm_sim.append(ddm_data)
    except Exception as e:
        print(str(e))
sec.disconnect()

  1%|█▎                                                                                                                                                                        | 4/505 [00:02<05:29,  1.52it/s]

"['commonstockdividendspersharedeclared'] not in index"


  1%|█▋                                                                                                                                                                        | 5/505 [00:03<05:29,  1.52it/s]

"['commonstockdividendspersharedeclared'] not in index"


  1%|██▎                                                                                                                                                                       | 7/505 [00:04<05:50,  1.42it/s]

...

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████    | 493/505 [06:33<00:09,  1.25it/s]

"['commonstockdividendspersharedeclared'] not in index"


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 502/505 [06:40<00:02,  1.28it/s]

"['commonstockdividendspersharedeclared'] not in index"


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [06:43<00:00,  1.25it/s]


In [129]:
simulation = pd.concat(ddm_sim).groupby(["year","quarter","ticker"]).mean().reset_index().drop("close",axis=1)

In [130]:
simulation = prices.merge(simulation,on=["year","quarter","ticker"]).dropna()

In [131]:
simulation

,date,open,high,low,close,volume,ticker,quarter,year,commonstockdividendspersharedeclared,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction,ddm_value
0,2016-01-04,148.0500,148.3200,145.4000,146.82,3277188,MMM,1,2016,1.744444,76.183950,0.005368,76.184189,-200.263899,86.355967,-0.496624,79.574702,74.005790
1,2016-01-05,146.8200,147.5000,145.6100,147.46,2688082,MMM,1,2016,1.744444,76.183950,0.005368,76.184189,-200.263899,86.355967,-0.496624,79.574702,74.005790
2,2016-01-06,145.5900,145.7600,143.4200,144.49,2997126,MMM,1,2016,1.744444,76.183950,0.005368,76.184189,-200.263899,86.355967,-0.496624,79.574702,74.005790
3,2016-01-07,142.5200,143.1300,140.6300,140.97,3553519,MMM,1,2016,1.744444,76.183950,0.005368,76.184189,-200.263899,86.355967,-0.496624,79.574702,74.005790
4,2016-01-08,141.3600,142.5000,140.2200,140.49,2663879,MMM,1,2016,1.744444,76.183950,0.005368,76.184189,-200.263899,86.355967,-0.496624,79.574702,74.005790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435481,2020-12-24,149.4000,149.7000,147.6100,149.25,283763,ZBH,4,2020,0.480000,114.461807,0.022082,114.509537,-28.365143,85.069831,0.119802,104.680392,95.622174
435482,2020-12-28,150.6100,151.5700,148.8000,148.89,683439,ZBH,4,2020,0.480000,114.461807,0.022082,114.509537,-28.365143,85.069831,0.119802,104.680392,95.622174
435483,2020-12-29,150.4300,153.3400,149.9119,152.45,735671,ZBH,4,2020,0.480000,114.461807,0.022082,114.509537,-28.365143,85.069831,0.119802,104.680392,95.622174
435484,2020-12-30,152.8500,154.7600,151.3700,152.22,427576,ZBH,4,2020,0.480000,114.461807,0.022082,114.509537,-28.365143,85.069831,0.119802,104.680392,95.622174


In [132]:
simulation["delta"] = (simulation["ddm_value"] - simulation["close"]) / simulation["close"]

In [143]:
simulation["ticker"].unique()

array(['MMM', 'ABT', 'ATVI', 'AAP', 'AES', 'A', 'APD', 'ALK', 'ALB',
       'ARE', 'LNT', 'ALL', 'MO', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT',
       'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ANTM', 'AON', 'AOS',
       'APA', 'AAPL', 'AMAT', 'ADM', 'AJG', 'AIZ', 'T', 'ADP', 'AVB',
       'AVY', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'BBY', 'TECH', 'BA',
       'BWA', 'BXP', 'BMY', 'BR', 'BRO', 'CHRW', 'CPB', 'COF', 'CAH',
       'CCL', 'CAT', 'CBOE', 'CNP', 'CF', 'SCHW', 'CVX', 'CB', 'CINF',
       'CTAS', 'CSCO', 'C', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL',
       'CMCSA', 'CMA', 'CAG', 'ED', 'GLW', 'COST', 'CTRA', 'CCI', 'CSX',
       'CMI', 'CVS', 'DHI', 'DE', 'DAL', 'XRAY', 'DVN', 'DFS', 'DG', 'D',
       'DPZ', 'DTE', 'DUK', 'EMN', 'ECL', 'EIX', 'ETR', 'EOG', 'EQIX',
       'EQR', 'ESS', 'EL', 'ES', 'RE', 'EXC', 'EXPE', 'EXPD', 'EXR',
       'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'F', 'BEN', 'FCX',
       'GPS', 'GRMN', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD',
       'GL', 

In [134]:
positions = 5
start_date = simulation["date"].min()
end_date = simulation["date"].max()
reqs = [1,3,5]
limit = 90
values = [True,False]
trades = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            date = start_date
            iterration_sim = simulation.copy()
            if not value:
                iterration_sim["delta"] = iterration_sim["delta"] * -1
            while date < end_date:
                try:
                    todays_recs = iterration_sim[iterration_sim["date"]==date]
                    todays_recs.sort_values("delta",ascending=False,inplace=True)
                    if todays_recs.index.size > 0:
                        offering = todays_recs.iloc[position]
                        if offering["delta"] > req:
                            trade = offering
                            ticker = trade["ticker"]
                            buy_price = trade["close"]
                            exits = iterration_sim[(iterration_sim["ticker"]==ticker) & (iterration_sim["date"]>date)].iloc[:limit]
                            exits["gains"] = (exits["close"] - buy_price) / buy_price
                            exit = exits.sort_values("gains",ascending=False).iloc[0]
                            trade["sell_price"] = exit["close"]
                            trade["sell_date"] = exit["date"]
                            date = exit["date"] + timedelta(days=1)
                            trade["value"] = value
                            trade["req"] = req
                            trade["position"] = position
                            trades.append(trade)
                        else:
                            date = date + timedelta(days=1)
                    else:
                        date = date + timedelta(days=1)
                except Exception as e:
                    print(str(e))
                    date = date+timedelta(days=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:13<00:00, 36.93s/it]


In [135]:
t = pd.DataFrame(trades)

In [136]:
t["delta"] =  (t["sell_price"] - t["close"]) / t["close"]

In [137]:
analysis = []
for value in tqdm(values):
    for req in tqdm(reqs):
        for position in range(positions):
            position_trades = t[(t["value"]==value) & (t["req"]==req) & (t["position"]==position)].sort_values("date")
            position_trades["delta"] = (position_trades["sell_price"] - position_trades["close"]) / position_trades["close"]
            initial = 100 / positions
            for delta in position_trades["delta"]:
                initial = initial * (1+delta)
            analysis.append({
                "value":value,
                "req":req,
                "position":position,
                "pv":initial
            })
            

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 176.35it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 55.56it/s]


In [138]:
complete = pd.DataFrame(analysis).pivot_table(index=["value","req"],columns="position",values="pv").reset_index()

In [141]:
complete["pv"] = [sum([row[1][i] for i in range(5)]) for row in complete.iterrows()]

In [142]:
complete.sort_values("pv",ascending=False)

position,value,req,0,1,2,3,4,pv
3,True,1,452.974299,547.119006,312.636576,430.462766,338.075287,2081.267935
4,True,3,452.974299,547.119006,312.636576,430.462766,338.075287,2081.267935
5,True,5,452.974299,371.976595,286.109810,742.628250,184.012979,2037.701933
0,False,1,86.003307,49.793940,43.159191,20.000000,20.000000,218.956438
1,False,3,46.029230,34.265138,20.000000,20.000000,20.000000,140.294368
2,False,5,47.973516,20.000000,20.000000,20.000000,20.000000,127.973516
